In [1]:
from abc import  ABC, abstractmethod
import aiohttp
import asyncio
import os
from fake_useragent import UserAgent
from typing import Dict, Any
import aiofiles
import json
from bs4 import BeautifulSoup
import re
import random

In [2]:
class Readable(ABC):
    def __init__(self): pass
    
    @abstractmethod
    def get(self): pass

class Writable(ABC):
    def __init__(self): pass
    
    @abstractmethod
    def put(self, data: Any): pass


In [ ]:
class WorkerWithHtml(Readable):
    
    def __init__(self):
        super().__init__()
        self._user = UserAgent().random
        
    @property
    def user(self):
        return  self._user

    async def get(self, 
                  session: aiohttp.ClientSession, 
                  url: str,
                  semaphore: asyncio.Semaphore= None,
                  accept: str= '*/*'):
        

        header = {'Accept' : accept, 
                  'User-Agent': self._user,
                  "Accept-Language": "ru,en;q=0.9",
                  "Accept-Encoding": "gzip, deflate, br",
                  "Connection": "keep-alive",
                  "Upgrade-Insecure-Requests": "1",
                  "Sec-Fetch-Dest": "document",
                  "Sec-Fetch-Mode": "navigate",
                  "Sec-Fetch-Site": "none",
                  "Sec-Fetch-User": "?1",
                  "Referer": "https://23met.ru/"}
        

        async def read_data_in_site():
            data = None
            if semaphore:
                async with semaphore:
                    # Чтение данных из сайта
                    async with session.get(url= url, 
                                        headers= header) as response:
                        data = await response.text()
                        
            else:
                # Чтение данных из сайта
                async with session.get(url= url, 
                                        headers= header) as response:
                    data = await response.text()
            return data

        data = await read_data_in_site()
        while BeautifulSoup(data, 'lxml').find('title').text == 'Слишком много запросов':
            print("Блокировка! Жду 2 минуты и пробую снова...")
            header['User-Agent'] = UserAgent().random
            await asyncio.sleep(120)
            data = await read_data_in_site()

        await asyncio.sleep(random.uniform(1, 3))
        return data

    async def _read_main_site(self, url: str, accept: str= "*/*"):

        # Забираем данные с основного сайта и кладем их в файла 
        data = None
        semaphore = asyncio.Semaphore(1)
        async with aiohttp.ClientSession() as session:
            data = await self.get(session= session, 
                                  url= url, 
                                  semaphore= semaphore,
                                  accept= accept)
        
        return data

In [4]:
class WorkerWithFiles(Readable, Writable):

    def __init__(self):
        super().__init__()

    async def get(self, path: str):
        data = None
        async with aiofiles.open(file= path) as file:
            data = await file.read()
        return data
    
    async def put(self, path: str, data: Any):
        async with aiofiles.open(file= path, mode= 'w') as file:
            await file.write(data)
    
    async def _put_json_file(self, path: str, data: Any):
        async with aiofiles.open(path, 'w') as file:
            json_str = json.dumps(obj= data, indent= 4, ensure_ascii= False) 
            await file.write(json_str)

In [ ]:
class Parser(ABC):
    def __init__(self, base_url: str):
        self.__file_worker = WorkerWithFiles()
        self.__html_worker = WorkerWithHtml()
        self.base_url = base_url


    @abstractmethod
    def parsing(): pass

    @property
    def user_agent(self):
        return self.__html_worker.user
    
    async def put_file(self, path: str, data: Any):
        await self.__file_worker.put(path= path, data= data)
    

    async def get_file(self, path: str):
        return await self.__file_worker.get(path= path)

    
    async def get_html(self, 
                       session: aiohttp.ClientSession,
                       url: str,
                       semaphore: asyncio.Semaphore= None,
                       accept: str= '*/*'):
        return await self.__html_worker.get(session= session, url= url, semaphore= semaphore, accept= accept)
    

    async def _read_main_site_and_save(self, 
                                       path_main_site: str, 
                                       file_name_for_main_site: str,
                                       accept: str= "*/*"):
        
        data = await self.__html_worker._read_main_site(self.base_url, 
                                                        accept= accept)
        
        # # сохраняет данные в файл
        # if BeautifulSoup(data, 'lxml').find(name= 'title').text == "Слишком много запросов":
        #     raise Exception("Вас заблокировали!")
        
        path= os.path.join(path_main_site, file_name_for_main_site)
        await self.put_file(path= path, data= data)
        

    async def _save_data_in_json_file(self, path: str, data: Any):
        await self.__file_worker._put_json_file(path, data)

In [6]:
class ParserSite_23MET(Parser):
    def __init__(self, base_url):
        super().__init__(base_url)

    async def __get_hrefs_for_next_sites(self,
                                         file_path: str):

        #сначало парсим основной сайт
        html_file = await self.get_file(path= file_path)
        soup = BeautifulSoup(markup= html_file, 
                             features= 'lxml')
        
        items_html_with_href= soup.find(name= 'nav', 
                                        attrs={"id" : "left-container", "class" : "left-container-mainpage-static"})\
                                  .find(name= 'ul')\
                                  .find_all(name= 'a')
        
        # формируем json с ссылками на сайты, внутри которых будут еще одни сайты с сылками на данные
        hrefs_next_sites = dict()
        for item in items_html_with_href:
            item: BeautifulSoup
            item_name = item.get_text()
            item_href = self.base_url + item.get('href').replace('/price', '')
            hrefs_next_sites[item_name] = item_href

        return hrefs_next_sites

    

    # async def __sub_get_hrefs_for_next_sites(self, 
    #                                          file_path: str):
    #     html_file = await self.get_file(path= file_path)
    
    
    @staticmethod
    def sanitize_filename(filename: str) -> str:
        # Заменяем все недопустимые символы на "_"
        return re.sub(r'[\\/:"*?<>|]+', '_', filename)
    
    async def _fetch_and_save_one(self, session, semaphore, url, path, detail_name, accept):
        """Скачивает и сразу сохраняет одну страницу."""

        async with semaphore:
            print(f"Начинаю обработку {detail_name}...")
            html = await self.get_html(session=session, url=url, accept=accept)

            if html:
                safe_name = self.sanitize_filename(filename= detail_name)
                file_path = os.path.join(path, f"{safe_name}.html")
                await self.put_file(path=file_path, data=html)
                print(f"Сохранил {detail_name} в {file_path}")
            else:
                print(f"Не удалось скачать {detail_name} с URL: {url}")




    async def parsing(self,
                      file_name_for_main_site= 'main_site.html',
                      accept= '*/*',
                      dir_name= None,
                      MAX_TASKS= 5): 
        
        path = os.getcwd()

        # создаю директорию, если есть dir_name
        if dir_name:
            path = os.path.join(path, dir_name)
            if not os.path.isdir(path):
                os.mkdir(path= path)
                print(f"Создал папку {dir_name} по пути {path}")
            else:
                print(f"Не стал создавать папку {dir_name}, т.к она уже существует!")
        
        try:
            await self._read_main_site_and_save(path_main_site= path, 
                                                file_name_for_main_site= file_name_for_main_site,
                                                accept= accept)

        except Exception as ex:
            print(ex)
            return ex


        # Получаем json с ссылками на сайты
        hrefs_next_sites = await self.__get_hrefs_for_next_sites(os.path.join(path, file_name_for_main_site)) 

        semaphore = asyncio.Semaphore(MAX_TASKS)
        
        # создадим отдельную директиву для сайтов
        SUBMAIN_DIR_NAME = 'submain_sites'
        dir_path = os.path.join(path, SUBMAIN_DIR_NAME) 
        counter= 0
        while os.path.isdir(dir_path):
            dir_path += str(counter)
            counter += 1
        os.mkdir(dir_path)

        async with aiohttp.ClientSession() as session:
            tasks = []
            for detail_name, submain_url in hrefs_next_sites.items():
                task = asyncio.create_task(
                    self._fetch_and_save_one(session, semaphore, submain_url, dir_path, detail_name, accept)
                )
                tasks.append(task)
            await asyncio.gather(*tasks)

In [7]:
worker = ParserSite_23MET("https://23met.ru/price")
await worker.parsing(dir_name= 'data')

Создал папку data по пути /home/ranil/Рабочий стол/Project/ПРОЕКТ/parsing/data
Начинаю обработку Арматура А1...
Начинаю обработку Арматура А1 оц....
Начинаю обработку Арматура А3...
Начинаю обработку Арматура А3 оц....
Начинаю обработку Арматура Ат800...
Сохранил Арматура А3 оц. в /home/ranil/Рабочий стол/Project/ПРОЕКТ/parsing/data/submain_sites/Арматура А3 оц..html
Начинаю обработку Арматура Ат1000...
Сохранил Арматура А3 в /home/ranil/Рабочий стол/Project/ПРОЕКТ/parsing/data/submain_sites/Арматура А3.html
Начинаю обработку Арматура СП...
Сохранил Арматура А1 оц. в /home/ranil/Рабочий стол/Project/ПРОЕКТ/parsing/data/submain_sites/Арматура А1 оц..html
Начинаю обработку Балка...
Сохранил Арматура Ат800 в /home/ranil/Рабочий стол/Project/ПРОЕКТ/parsing/data/submain_sites/Арматура Ат800.html
Начинаю обработку Балка б/у...
Сохранил Арматура А1 в /home/ranil/Рабочий стол/Project/ПРОЕКТ/parsing/data/submain_sites/Арматура А1.html
Начинаю обработку Бочонок...
Сохранил Арматура Ат1000 в /hom

CancelledError: 